## Pancakeswap 彩票规则

1. 彩票价格：1 张彩票 - 大约 5 美金的 CAKE
2. 单个用户购彩限制：没有总张数限制，但是单次最多只能购买 100 张
3. 购买一张彩票，用户即获得一个随机的 6 位数组合。每个数字都在 0-9 范围内，例如：“1-9-3-2-0-4”。从左边开始匹配数字来赢取奖金
4. 购票者购买该回合彩票所支付的 CAKE 将全部注入奖池
5. 在每一回合之后，如果其中一个奖池中无人中奖，则该组中无人领取的 CAKE 将滚存到下一回合并在奖池中重新分配
6. 每张中奖彩票将均分对应分级奖池中的奖金
7. 分级奖池如下：


|  分级奖池      |    CAKE 分配  |
|  ------------| ----  |
| 第一个数字匹配  | 2% |
| 前两个数字匹配  | 3% |
| 前三个数字匹配  | 5% |
| 前四个数字匹配  | 10% |
| 前五个数字匹配  | 20% |
| 六个数字全部匹配 | 40% |
| 销毁部分       | 20% |


## 数学期望
根据以上规则，在给定的 m 个随机数且每个随机数字有 r 种可能的条件下按序匹配 n 个数字的概率：
$$P = \frac{1}{r^n}, n <= m$$

数学期望: 
$$E(X)=\sum_{i=1}^{n}{X_i}{P_i}$$
其中 $X_i$ 为对应奖池可平分得到的奖金，$P_i$ 为对应奖池中奖概率

In [727]:
# 不考虑上次滚存奖金，以单次总奖金池 $5000, 总开奖数 1000 为例
import random

r = 10
draws = 1000
ticket_value = 5
total = ticket_value * draws

def calc_probability(n: int) -> int:
    return 1 / (r ** n)

# 各等级奖金池分配
prize_pool_allocation = {1: 0.02, 2: 0.03, 3: 0.05, 4: 0.1, 5: 0.2, 6: 0.4}

# 各等级中奖概率
probabilities = {}
for n, _ in prize_pool_allocation.items():
    probabilities[n] = calc_probability(n)

# 各等级奖金池平均中奖人数
shares = {}
for n, p in probabilities.items():
    shares[n] = draws * p

# 数学期望
E = 0
for n, p in probabilities.items():
    E += total*prize_pool_allocation[n] / shares[n] * p

print(f"E={E}")  # E=4


E=4.0


## 模拟测试

In [734]:

import random
from typing import List, Tuple


def lucky_number() -> List[int]:
    return [random.randint(0, 9) for _ in range(6)]


def check_ticket_matched(final: List[int], ticket: List[int]):
    i = 0
    while i < len(ticket):
        if (ticket[i] != final[i]):
            break
        i += 1
    return i

def my_draw(rollover_prize):
    prize_pool_allocation = {1: 0.02, 2: 0.03, 3: 0.05, 4: 0.1, 5: 0.2, 6: 0.4}
    total = ticket_value * draws + rollover_prize
    final = lucky_number()
    stats = {}

    def match(ticket) -> int:
        matched = check_ticket_matched(final, ticket)
        if matched in stats:
            stats[matched] += 1
        else:
            stats[matched] = 1
        return matched

    # 其他参与者开奖
    for i in range(draws - 1):
        ticket = lucky_number()
        match(ticket)

    # 个人开奖
    my_ticket = lucky_number()
    my_prize = 0
    matched_count = match(my_ticket)
    count = 1
    if matched_count in stats:
        count = stats[matched_count]
    if matched_count in prize_pool_allocation:
        my_prize = prize_pool_allocation[matched_count] * total / count
    else:
        my_prize = 0

    # 下一回合滚存资金
    current_rollover_prize = 0
    for rank, _ in stats.items():
        if rank in prize_pool_allocation:
            prize_pool_allocation.pop(rank)
    for _, percentage in prize_pool_allocation.items():
        current_rollover_prize += percentage * total

    return my_prize, current_rollover_prize

draws = 1000
ticket_value = 5

try_count = 500
rollover_prize = 0
my_total_prize = 0

for i in range(try_count):
    prize, current_rollover_prize = my_draw(rollover_prize)
    rollover_prize = current_rollover_prize
    my_total_prize += prize

mean_prize = my_total_prize / try_count
print(mean_prize)


1.0297188606832761


In [ ]:
import random
from typing import List, Tuple


def calc_probability(n: int) -> int:
    return 1 / (10 ** n)


def lucky_number() -> List[int]:
    return [random.randint(0, 9) for _ in range(6)]


def check_ticket_matched(final: List[int], ticket: List[int]):
    i = 0
    while i < len(ticket):
        if (ticket[i] != final[i]):
            break
        i += 1
    return i


draws = 1000
ticket_value = 5


def draw(rollover_prize):
    prize_pool_allocation = {1: 0.02, 2: 0.03, 3: 0.05, 4: 0.1, 5: 0.2, 6: 0.4}
    total_prize = ticket_value * draws + rollover_prize
    final = lucky_number()
    stats = {}

    # 模拟开奖
    for i in range(draws):
        ticket = lucky_number()
        matched_count = check_ticket_matched(final, ticket)
        if matched_count in stats:
            stats[matched_count] += 1
        else:
            stats[matched_count] = 1

    # 数学期望
    E = 0
    remain_pool = range(1, 7)
    for matched_count, count in stats.items():
        if matched_count in prize_pool_allocation:
            E += prize_pool_allocation[matched_count] * \
                total_prize / count * calc_probability(matched_count)
            prize_pool_allocation.pop(matched_count)

    # 滚存资金
    current_rollover_prize = 0
    for _, percent in prize_pool_allocation.items():
        current_rollover_prize += percent * total_prize
    return (E, current_rollover_prize)
